In [1]:
#Import required libraries
import ee
import geemap

In [2]:
#Visualize map

Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [3]:
#Import NLCD landcover dataset

NLCD2016 = ee.Image('USGS/NLCD/NLCD2016').select('landcover')
Map.addLayer(NLCD2016, {}, 'NLCD 2016')

In [4]:
#Select our region of interest

region = ee.Geometry.Rectangle([-84.42291, 30.55397, -84.14871, 30.08362])

In [5]:
#clip the NLCD dataset to the region  of our interest and it to the map

metadata = NLCD2016.clip(region)
region2 = metadata.geometry()
Map.addLayer(region2,{},'NLCD')
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [6]:
#Collect metadata information from the clipped dataset

metadata.get('2016on_bas').getInfo()
metadata

In [7]:
#Look for NAIP dataset and filterbound to the region of our interest

dataset = ee.ImageCollection('USDA/NAIP/DOQQ').filter(ee.Filter.date('2017-01-01', '2018-12-31')).filterBounds(region)

In [8]:
#Calculate the median to get an image from the image collection

new=dataset.median()

In [9]:
#add the NAIP layer to the map

Map.addLayer(new,{},'NAIP')
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [10]:
#get the information of the classes in NLCD dataset clipped to the region of our interest

raw_class_values = metadata.get('landcover_class_values').getInfo()
print(raw_class_values)

[11, 12, 21, 22, 23, 24, 31, 41, 42, 43, 51, 52, 71, 72, 73, 74, 81, 82, 90, 95]


In [11]:
#rename the classes name from 0 to 19 to make the analysis easier

n_classes = len(raw_class_values)
new_class_values = list(range(0, n_classes))
new_class_values

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

In [12]:
#get the information of the colour palette of the classes of NLCD dataset

class_palette = metadata.get('landcover_class_palette').getInfo()
print(class_palette)

['476ba1', 'd1defa', 'decaca', 'd99482', 'ee0000', 'ab0000', 'b3aea3', '68ab63', '1c6330', 'b5ca8f', 'a68c30', 'ccba7d', 'e3e3c2', 'caca78', '99c247', '78ae94', 'dcd93d', 'ab7028', 'bad9eb', '70a3ba']


In [13]:
#map the new classes name and the respective colour palette to the dataset

nlcd = metadata.remap(raw_class_values, new_class_values).select(['remapped'], ['landcover'])
nlcd = nlcd.set('landcover_class_values', new_class_values)
nlcd = nlcd.set('landcover_class_palette', class_palette)

In [14]:
#add new layer to the map with name NLCD

Map.addLayer(nlcd, {}, 'NLCD')
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [15]:
#add random sample points (90000) on the dataset

points = metadata.sample(**{
    'region': dataset.geometry(),
    'scale': 30,
    'numPixels': 90000,
    'seed': 0,
    'geometries': True  # Set this to False to ignore geometries
})

Map.addLayer(points, {}, 'training', False)

In [16]:
#print the size of the sample points

print(points.size().getInfo())

69722


In [17]:
print(points.first().getInfo())

{'type': 'Feature', 'geometry': {'geodesic': False, 'type': 'Point', 'coordinates': [-84.15508746136601, 30.28997985666014]}, 'id': '0', 'properties': {'landcover': 21}}


In [18]:
bands = bands = ['R', 'G', 'B', 'N']

# This property of the table stores the land cover labels.
label = 'landcover'

# Overlay the points on the imagery to get training.
sample = new.select(bands).sampleRegions(**{
  'collection': points,
  'properties': [label],
  'scale': 30
})

# Adds a column of deterministic pseudorandom numbers. 
sample = sample.randomColumn()

split = 0.7

training = sample.filter(ee.Filter.lt('random', split))
validation = sample.filter(ee.Filter.gte('random', split))

In [19]:
#select the classifier to train the model

classifier = ee.Classifier.smileRandomForest(10).train(training, label, bands)

In [20]:
result = new.select(bands).classify(classifier)

# # Display the clusters with random colors.
Map.addLayer(result.randomVisualizer(), {}, 'classfied')
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [21]:
class_values = nlcd.get('landcover_class_values').getInfo()
print(class_values)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]


In [22]:
class_palette = nlcd.get('landcover_class_palette').getInfo()
print(class_palette)

['476ba1', 'd1defa', 'decaca', 'd99482', 'ee0000', 'ab0000', 'b3aea3', '68ab63', '1c6330', 'b5ca8f', 'a68c30', 'ccba7d', 'e3e3c2', 'caca78', '99c247', '78ae94', 'dcd93d', 'ab7028', 'bad9eb', '70a3ba']


In [23]:
landcover = result.set('classification_class_values', class_values)
landcover = landcover.set('classification_class_palette', class_palette)

In [24]:
Map.addLayer(landcover, {}, 'Land cover')
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [25]:
print('Change layer opacity:')
cluster_layer = Map.layers[-1]
cluster_layer.interact(opacity=(0, 1, 0.1))

Change layer opacity:


Box(children=(FloatSlider(value=1.0, description='opacity', max=1.0),))

In [26]:
Map.add_legend(builtin_legend='NLCD')
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [27]:

#test the train_accuracy of the model
train_accuracy = classifier.confusionMatrix()

In [ ]:
train_accuracy.getInfo().head(10)

In [ ]:
train_accuracy.accuracy().getInfo()

In [ ]:
train_accuracy.kappa().getInfo()

In [ ]:
train_accuracy.producersAccuracy().getInfo().head(10)

In [ ]:
train_accuracy.consumersAccuracy().getInfo().head(10)

In [ ]:
validated = validation.classify(classifier)

In [ ]:
validated.first().getInfo()

In [ ]:
test_accuracy = validated.errorMatrix('landcover', 'classification')

In [ ]:
test_accuracy.getInfo().head(10)

In [ ]:
#get the test accuracy of the model

test_accuracy.accuracy().getInfo()

In [ ]:
test_accuracy.kappa().getInfo()

In [ ]:
test_accuracy.producersAccuracy().getInfo().head(10)

In [ ]:
import csv
import os

out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
training_csv = os.path.join(out_dir, 'train_accuracy_NAIP.csv')
testing_csv = os.path.join(out_dir, 'test_accuracy_NAIp.csv')

with open(training_csv, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(train_accuracy.getInfo())
    
with open(testing_csv, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(test_accuracy.getInfo())

In [ ]:
landcover = landcover.remap(new_class_values, raw_class_values).select(['remapped'], ['classification'])

In [ ]:
landcover = landcover.set('classification_class_values', raw_class_values)
landcover = landcover.set('classification_class_palette', class_palette)

In [ ]:
Map.addLayer(landcover, {}, 'Final land cover')
Map

In [ ]:
import os
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
out_file = os.path.join(out_dir, 'landcover.tif')

In [ ]:
#save the classified map to the google drive

geemap.ee_export_image(landcover, filename=out_file, region=region,fileDimensions=5000,shardSizwe=100,fileFormat="GeoTiFF",scale=1)